# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [1]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn_light.yml
    !mamba env update -n base -f environment_amn_light.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Functions.Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(V, Stats, model): 
    # printing Stats
    print("R2 = %.2f (+/- %.2f) Constraint = %.2f (+/- %.2f)" % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    Vout = tf.convert_to_tensor(np.float32(model.Y))
    Loss_norm, dLoss = Loss_Vout(V, model.Pout, Vout)
    print('Loss Targets', np.mean(Loss_norm))
    Loss_norm, dLoss = Loss_SV(V, model.S)
    print('Loss SV', np.mean(Loss_norm))
    Vin = tf.convert_to_tensor(np.float32(model.X))
    Pin = tf.convert_to_tensor(np.float32(model.Pin))
    if Vin.shape[1] == model.S.shape[1]: # special case
        Vin  = tf.linalg.matmul(Vin, tf.transpose(Pin), b_is_sparse=True)
    Loss_norm, dLoss = Loss_Vin(V, model.Pin, Vin, model.mediumbound)
    print('Loss Vin bound', np.mean(Loss_norm))
    Loss_norm, dLoss = Loss_Vpos(V, model)
    print('Loss V positive', np.mean(Loss_norm))

['README.md', 'Duplicate_Model.ipynb', 'Build_Model_Dense.ipynb', 'Dataset_experimental', '.ipynb_checkpoints', '.git', 'Build_Model_RC.ipynb', 'environment_amn_light.yml', 'Build_Experimental.ipynb', 'Reservoir', 'Build_Model_MM.ipynb', 'Dataset_model', 'Figures.ipynb', 'Result', 'Figures', '.gitignore', 'LICENSE', 'Build_Model_ANN.ipynb', 'Build_Dataset.ipynb', 'Dataset_input', 'Functions', '__pycache__', 'old', 'environment_amn.yml', 'Build_Model_AMN.ipynb', 'Build_Model.ipynb', '.DS_Store']


## (1) Mechanistic Model: examples with non trainable mechanistic model with FBA simulated training sets or experimental datasets

In [2]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using tiny5 simulation training sets with EB (or UB) bounds

# What you can change
seed = 1
np.random.seed(seed=seed)  
trainname = 'tiny5_EB'
write_loss = False # Writes an output file containing loss history mean and std over timesteps
write_targets = False # Writes an output file containing true and predicted targets
# End of What you can change

# Create model and run GD
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['biomass'], 
              model_type = 'MM_LP',
              timestep = int(1.0e3), learn_rate = 1.0, decay_rate = 0.5)
model.printout()

loss_outname = DIRECTORY + "Result/" + model.model_type + "_Loss_" + trainname + ".csv" if write_loss else None
model.loss_outfile = loss_outname
targets_outname = DIRECTORY + "Result/" + model.model_type + "_Targets_" + trainname + ".csv" if write_targets else None
model.targets_outfile = targets_outname

if model.model_type is 'MM_QP':
    Ypred, Stats = MM_QP(model, verbose=True)
if model.model_type is 'MM_LP':
    Ypred, Stats = MM_LP(model, verbose=True)

# Printing results
printout(Ypred, Stats, model)

./Dataset_model/tiny5_EB.npz


SystemExit: parameter file not found

/home/lfaure/miniconda3/envs/AMN/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using E. coli core simulation training sets and EB (or UB) bounds

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # the training set file name
size = 10 # number of runs must be lower than the number of element in trainname
timestep = int(1.0e4) # LP 1.0e4 QP 1.0e5
learn_rate = 0.3 # LP 0.3 QP 1.0
decay_rate = 0.9 # only in QP, UB 0.333 EB 0.9
write_loss = False # Writes an output file containing loss history mean and std over timesteps
write_targets = False # Writes an output file containing true and predicted targets
# End of What you can change

# Create model and run GD for X and Y randomly drawn from trainingfile
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = 'MM_LP', 
              timestep = timestep, 
              learn_rate = learn_rate, 
              decay_rate = decay_rate)

# Select a random subset of the training set (of specified size)

ID = np.random.choice(model.X.shape[0], size, replace=False)
model.X, model.Y = model.X[ID,:], model.Y[ID,:]
if model.mediumbound == 'UB':
    model.b_ext = model.b_ext[ID,:]
if model.mediumbound == 'EB':
    model.b_int = model.b_int[ID,:]

# Prints a summary of the model before running

model.printout()

# Make names for output file (if 'write_' flag is set to true)

loss_outname = DIRECTORY + "Result/" + model.model_type + "_Loss_" + \
trainname + ".csv" if write_loss else None
model.loss_outfile = loss_outname
targets_outname = DIRECTORY + "Result/" + model.model_type + "_Targets_" + \
trainname + ".csv" if write_targets else None
model.targets_outfile = targets_outname

# Runs the appropriate method

if model.model_type is 'MM_QP':
    Ypred, Stats = MM_QP(model, verbose=True)
if model.model_type is 'MM_LP':
    Ypred, Stats = MM_LP(model, verbose=True)

# Printing results
printout(Ypred, Stats, model)

training file: ./Dataset_model/e_coli_core_UB
model type: MM_LP
model scaler: 0.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 10000
training set size (10, 20) (10, 1)
LP-Loss 1 0.25261024 0.06849076
LP-Loss 10 0.2294474 0.06633821
LP-Loss 100 0.0808966 0.03674851
LP-Loss 1000 0.00023186656 0.0002091549
LP-Loss 2000 3.8031656e-05 2.6454396e-05
LP-Loss 3000 1.6315684e-05 1.6629287e-05
LP-Loss 4000 9.442818e-06 1.449519e-05
LP-Loss 5000 7.2193407e-06 1.2035626e-05
LP-Loss 6000 5.5323576e-06 9.809663e-06
LP-Loss 7000 4.2318406e-06 7.880076e-06
LP-Loss 8000 3.2548737e-06 6.2515983e-06
LP-Loss 9000 2.3623904e-06 4.757077e-06
LP-Loss 10000 1.6831349e-06 3.5644443e-06
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (10, 1) (10, 1) (10, 1) (10, 1) (10, 154) (10, 312)
R2 = 1.00 (+/- 0.00) Constraint = 0.00 (+/- 0.00)
Loss Targets 0.001244092
Loss SV 8.0880665e-07
Loss Vin bound 1.0041353e-05
Loss V positive 2.924077e-06


In [ ]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using E. coli core simulation training sets and EB (or UB) bounds

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_EB' # the training set file name
size = 10 # number of runs must be lower than the number of element in trainname
timestep = int(1.0e6) # LP 1.0e4 QP 1.0e5
learn_rate = 1.0 # LP 0.3 QP 1.0
decay_rate = 0.9 # only in QP, UB 0.333 EB 0.9
write_loss = False # Writes an output file containing loss history mean and std over timesteps
write_targets = False # Writes an output file containing true and predicted targets
# End of What you can change

# Create model and run GD for X and Y randomly drawn from trainingfile
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = 'MM_QP', 
              timestep = timestep, 
              learn_rate = learn_rate, 
              decay_rate = decay_rate)

# Select a random subset of the training set (of specified size)
ID = np.random.choice(model.X.shape[0], size, replace=False)
model.X, model.Y = model.X[ID,:], model.Y[ID,:]
if model.mediumbound == 'UB':
    model.b_ext = model.b_ext[ID,:]
if model.mediumbound == 'EB':
    model.b_int = model.b_int[ID,:]

# Prints a summary of the model before running
model.printout()

# Make names for output file (if 'write_' flag is set to true)
loss_outname = DIRECTORY + "Result/" + model.model_type + "_Loss_" \
+ trainname + ".csv" if write_loss else None
model.loss_outfile = loss_outname
targets_outname = DIRECTORY + "Result/" + model.model_type + "_Targets_" \
+ trainname + ".csv" if write_targets else None
model.targets_outfile = targets_outname

# Runs the appropriate method
if model.model_type is 'MM_QP':
    Ypred, Stats = MM_QP(model, loss_outfile=loss_outname, 
                         targets_outfile= targets_outname,
                         verbose=True)
if model.model_type is 'MM_LP':
    Ypred, Stats = MM_LP(model, loss_outfile=loss_outname, 
                         targets_outfile= targets_outname,
                         verbose=True)
# Printing results
printout(Ypred, Stats, model)

In [ ]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using E. coli core simulation training sets
# with initial X close to solution Y and EB (or UB) bounds

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # the training set file name (EB or UB)
size = 10 # number of runs must be lower than the number of element in trainname
timestep = int(1.0e4) # LP 1.0e4 QP 1.0e5
learn_rate = 1.0 # LP 0.3 QP 1.0
decay_rate = 0.9 # only in QP, UB 0.333 EB 0.9 # try 0.5
write_loss = False # Writes an output file containing loss history mean and std over timesteps
write_targets = False # Writes an output file containing true and predicted targets
model_type = 'MM_QP'
# End of What you can change

# Draw X and Y randomly drawn from trainingfile
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = model_type, 
              timestep = timestep,
              learn_rate = learn_rate,
              decay_rate = decay_rate)
ID = np.random.choice(model.X.shape[0], size, replace=False)
model.X, model.Y, model.Yall = model.X[ID,:], model.Y[ID,:],  model.Yall[ID,:]
if model.mediumbound == 'UB':
    model.b_ext = model.b_ext[ID,:]
if model.mediumbound == 'EB':
    model.b_int = model.b_int[ID,:]
model.printout()

# Create model and run GD X = normal distribution around solution
Ypred, Stats = {}, {}
noise_set = [0.5, 0.1, 0.01, 0.005, 0.001, 0.0005]
for noise in noise_set:
    
    loss_outname = DIRECTORY + "Result/" + str(noise) + "_noise_" + \
    model.model_type + "_Loss_" + trainname + ".csv" if write_loss else None
    model.loss_outfile = loss_outname
    targets_outname = DIRECTORY + "Result/" + str(noise) + "_noise_" \
    + model.model_type + "_Targets_" + trainname + ".csv" if write_targets else None
    model.targets_outfile = targets_outname
    
    
    model.X = np.random.normal(model.Yall, noise*model.Yall)
    print('noise', noise)
    if model.model_type is 'MM_QP':
        Ypred[noise], Stats[noise] = MM_QP(model, verbose=True)
    if model.model_type is 'MM_LP':
        Ypred[noise], Stats[noise] = MM_LP(model, verbose=True)
    
    printout(Ypred[noise], Stats[noise], model)

training file: ./Dataset_model/e_coli_core_UB
model type: MM_QP
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 10000
training set size (10, 20) (10, 1)
noise 0.5
GD-Loss 1 2.3497357 0.58461136
GD-Loss 10 2.1286168 0.66098446
GD-Loss 100 0.68807036 0.15949886
GD-Loss 1000 0.17094067 0.048883222
GD-Loss 2000 0.112243116 0.03465901
GD-Loss 3000 0.08686058 0.028503802
GD-Loss 4000 0.071525335 0.024566654
GD-Loss 5000 0.061294593 0.021584878
GD-Loss 6000 0.05490441 0.019376915
GD-Loss 7000 0.048432 0.01873374
GD-Loss 8000 0.044350315 0.016383402
GD-Loss 9000 0.040097177 0.015305694
GD-Loss 10000 0.037610233 0.014153577
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (10, 1) (10, 1) (10, 1) (10, 1) (10, 154) (10, 158)
R2 = 1.00 (+/- 0.00) Constraint = 0.10 (+/- 0.03)
Loss Targets 0.001111585
Loss SV 0.034170542
Loss Vin bound 0.20221159
Loss V positive 0.0014855479
noise 0.1
GD-Loss 1 0.47501326 0.122467846
GD-Loss 10 0.38130006 0.13565588
GD-Loss 100 0.1461